In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
import os

os.environ["GOOGLE_API_KEY"] = ""

In [3]:
llm = ChatGoogleGenerativeAI(model = "gemini-pro")

In [5]:
from crewai import Crew,Agent,Task

In [9]:
from crewai import Crew, Agent, Task
from crewai_tools import tool
@tool("Request User Feedback")
def request_user_feedback(conversation: str) -> str:
    """This tool presents a text for conversation to the user and requests feedback.

    Args:
        conversation (str): The conversation text created by the agent.

    Returns:
        str: The feedback provided by the user.
    """
    feedback = input(conversation)
    return feedback

humanizer = Agent(
    role="Medical Assistant Chatbot",
    goal="To assist patients by asking them relevant questions about their symptoms and forming a detailed report for the doctor.",
    backstory="A friendly and empathetic virtual assistant designed to help patients communicate their symptoms effectively.",
    llm=llm,
    tools=[request_user_feedback],
    allow_delegation=False,
    verbose=True
)

# Define the humanize task
humanize = Task(
    description=(
        "You have to follow each and every step to ensure that the patient's symptoms are accurately recorded and reported to the doctor."
        "1. Greet the patient and introduce yourself."
        "2. Ask for the patient's name and age."
        "3. Inquire about the main symptom the patient is experiencing."
        "4. Ask specific questions based on the reported symptom."
        "5. Assess the pain intensity and characteristics if applicable."
        "6. Inquire about any medication taken and its effects."
        "7. Determine the duration of the symptoms."
        "8. Ask for any additional relevant information."
        "9. Thank the patient and provide information on the next steps."
    ),
    expected_output="A detailed report containing the patient's symptoms, pain assessment, medication details, symptom duration, and any additional information.",
    agent=humanizer,
)


In [10]:
crew = Crew(agents=[humanizer], tasks=[humanize],verbose=2)
result = crew.kickoff()

2024-06-26 15:21:25,327 - 11500 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-06-26 15:21:25][DEBUG]: == Working Agent: Medical Assistant Chatbot
 [2024-06-26 15:21:25][INFO]: == Starting Task: You have to follow each and every step to ensure that the patient's symptoms are accurately recorded and reported to the doctor.1. Greet the patient and introduce yourself.2. Ask for the patient's name and age.3. Inquire about the main symptom the patient is experiencing.4. Ask specific questions based on the reported symptom.5. Assess the pain intensity and characteristics if applicable.6. Inquire about any medication taken and its effects.7. Determine the duration of the symptoms.8. Ask for any additional relevant information.9. Thank the patient and provide information on the next steps.


> Entering new CrewAgentExecutor chain...
Action: Request User Feedback
Action Input: {"conversation": "Hello! I'm here to help you communicate your symptoms to the doctor. May I know your name and age?"} 

My name is Sneh and I am 20 years old

Thought:
Action: Request User F

In [11]:
print(result)

Patient Name: Sneh Age: 20 Symptoms: - High fever (99 degrees Fahrenheit) - Cold - Cough Pain Assessment: None provided Medication: Paracetamol, effective for a few hours Symptom Duration: 2 days Additional Information: None
